In [ ]:
from tensorflow.keras import layers
from tensorflow import keras
from ast import literal_eval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle

!cp /content/drive/MyDrive/KaggleAPI/kaggle.json ~/.kaggle/kaggle.json

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download -d spsayakpaul/arxiv-paper-abstracts

Dataset URL: https://www.kaggle.com/datasets/spsayakpaul/arxiv-paper-abstracts
License(s): CC0-1.0
 99% 44.0M/44.6M [00:03<00:00, 22.1MB/s]
100% 44.6M/44.6M [00:03<00:00, 14.1MB/s]


In [ ]:
! unzip /content/arxiv-paper-abstracts.zip

Archive:  /content/arxiv-paper-abstracts.zip
  inflating: arxiv_data.csv          
  inflating: arxiv_data_210930-054931.csv  


In [ ]:
arxiv_data = pd.read_csv('arxiv_data_210930-054931.csv')

In [ ]:
arxiv_data.head(10)

,terms,titles,abstracts
0,['cs.LG'],Multi-Level Attention Pooling for Graph Neural...,Graph neural networks (GNNs) have been widely ...
1,"['cs.LG', 'cs.AI']",Decision Forests vs. Deep Networks: Conceptual...,Deep networks and decision forests (such as ra...
2,"['cs.LG', 'cs.CR', 'stat.ML']",Power up! Robust Graph Convolutional Network v...,Graph convolutional networks (GCNs) are powerf...
3,"['cs.LG', 'cs.CR']",Releasing Graph Neural Networks with Different...,With the increasing popularity of Graph Neural...
4,['cs.LG'],Recurrence-Aware Long-Term Cognitive Network f...,Machine learning solutions for pattern classif...
5,"['cs.LG', 'stat.ML']",Lifelong Graph Learning,Graph neural networks (GNNs) are powerful mode...
6,['cs.LG'],Bayesian graph convolutional neural networks v...,"Deep learning models, such as convolutional ne..."
7,"['cs.LG', 'stat.ML']",Understanding and Resolving Performance Degrad...,A Graph Convolutional Network (GCN) stacks sev...
8,['cs.LG'],Edge-augmented Graph Transformers: Global Self...,Transformer neural networks have achieved stat...
9,"['cs.LG', 'cs.AI']",Variational Graph Normalized Auto-Encoders,Link prediction is one of the key problems for...


In [ ]:
arxiv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56181 entries, 0 to 56180
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   terms      56181 non-null  object
 1   titles     56181 non-null  object
 2   abstracts  56181 non-null  object
dtypes: object(3)
memory usage: 1.3+ MB


In [ ]:
arxiv_data.describe()

,terms,titles,abstracts
count,56181,56181,56181
unique,3402,41105,41115
top,['cs.CV'],Deep Reinforcement Learning: An Overview,Mesh is a powerful data structure for 3D shape...
freq,18719,7,7


In [ ]:
arxiv_data.duplicated().sum()

15054

In [ ]:
arxiv_data.drop_duplicates(inplace=True)
arxiv_data.reset_index(drop= True,inplace = True)

In [ ]:
arxiv_data.describe()

,terms,titles,abstracts
count,41127,41127,41127
unique,3402,41105,41115
top,['cs.CV'],Fast Convergence of DETR with Spatially Modula...,Modern implicit generative models such as gene...
freq,13160,2,2


# **RECOMMENDATION SYSTEM**

In [ ]:
titles = arxiv_data['titles']

In [ ]:
titles

0        Multi-Level Attention Pooling for Graph Neural...
1        Decision Forests vs. Deep Networks: Conceptual...
2        Power up! Robust Graph Convolutional Network v...
3        Releasing Graph Neural Networks with Different...
4        Recurrence-Aware Long-Term Cognitive Network f...
                               ...                        
41122    An experimental study of graph-based semi-supe...
41123    Bayesian Differential Privacy through Posterio...
41124    Mining Spatio-temporal Data on Industrializati...
41125    Wav2Letter: an End-to-End ConvNet-based Speech...
41126                          Generalized Low Rank Models
Name: titles, Length: 41127, dtype: object

In [ ]:
abstracts = arxiv_data['abstracts']
abstracts

0        Graph neural networks (GNNs) have been widely ...
1        Deep networks and decision forests (such as ra...
2        Graph convolutional networks (GCNs) are powerf...
3        With the increasing popularity of Graph Neural...
4        Machine learning solutions for pattern classif...
                               ...                        
41122    The volume of data generated by internet and s...
41123    Differential privacy formalises privacy-preser...
41124    Despite the growing availability of big data i...
41125    This paper presents a simple end-to-end model ...
41126    Principal components analysis (PCA) is a well-...
Name: abstracts, Length: 41127, dtype: object

In [ ]:
!pip install -U -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 64.9 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
if not isinstance(titles, list):
    titles_list = titles.tolist()
else:
    titles_list = titles

embeddings = model.encode(titles_list)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
if not isinstance(abstracts, list):
    abstracts_list = abstracts.tolist()
else:
    abstracts_list = abstracts

abstracts_embeddings = model.encode(abstracts_list)

In [ ]:
embeddings

array([[ 0.06643406, -0.04954598,  0.06388087, ...,  0.00106302,
        -0.12156384, -0.06962777],
       [ 0.09212259, -0.07606942,  0.06572868, ..., -0.08565166,
        -0.09266548,  0.00725291],
       [-0.08162685,  0.02428936,  0.01888745, ...,  0.00806161,
        -0.05129537, -0.05873998],
       ...,
       [ 0.01227976, -0.08568838, -0.02782773, ..., -0.05257973,
        -0.10806683,  0.07843307],
       [-0.07258198, -0.12690918, -0.00535554, ...,  0.03597706,
        -0.03986146, -0.05971029],
       [ 0.00768872, -0.1012418 ,  0.08909857, ..., -0.08199867,
        -0.05649745,  0.09007055]], dtype=float32)

In [ ]:
for i in range(len(embeddings)):
  print("Sentences: ", titles[i])
  print("Embedding: ", len(embeddings[i]))
  print(" ")
  if i == 5:
    break

Sentences:  Multi-Level Attention Pooling for Graph Neural Networks: Unifying Graph Representations with Multiple Localities
Embedding:  384
 
Sentences:  Decision Forests vs. Deep Networks: Conceptual Similarities and Empirical Differences at Small Sample Sizes
Embedding:  384
 
Sentences:  Power up! Robust Graph Convolutional Network via Graph Powering
Embedding:  384
 
Sentences:  Releasing Graph Neural Networks with Differential Privacy Guarantees
Embedding:  384
 
Sentences:  Recurrence-Aware Long-Term Cognitive Network for Explainable Pattern Classification
Embedding:  384
 
Sentences:  Lifelong Graph Learning
Embedding:  384
 


In [ ]:
for i in range(len(abstracts_embeddings)):
  print("Sentences: ", abstracts[i])
  print("Embedding: ", len(abstracts_embeddings[i]))
  print(" ")
  if i == 5:
    break

Sentences:  Graph neural networks (GNNs) have been widely used to learn vector
representation of graph-structured data and achieved better task performance
than conventional methods. The foundation of GNNs is the message passing
procedure, which propagates the information in a node to its neighbors. Since
this procedure proceeds one step per layer, the range of the information
propagation among nodes is small in the lower layers, and it expands toward the
higher layers. Therefore, a GNN model has to be deep enough to capture global
structural information in a graph. On the other hand, it is known that deep GNN
models suffer from performance degradation because they lose nodes' local
information, which would be essential for good model performance, through many
message passing steps. In this study, we propose multi-level attention pooling
(MLAP) for graph-level classification tasks, which can adapt to both local and
global structural information in a graph. It has an attention pooling l

In [ ]:
def recommendation(input_paper):
    # Calculate cosine similarity scores between the embeddings of input_paper and all papers in the dataset.
    input_embedding = model.encode(input_paper)
    cosine_scores = cosine_similarity(embeddings, input_embedding.reshape(1, -1))

    # Get the indices of the top-k most similar papers based on cosine similarity.
    top_similar_papers_indices = np.argsort(cosine_scores, axis=0)[-5:][::-1].flatten()

    # Retrieve the titles of the top similar papers.
    papers_list = [titles[i] for i in top_similar_papers_indices]

    return papers_list

In [ ]:
def recommendation_abstracts(input_paper):
    input_embedding = model.encode(input_paper)
    cosine_scores = cosine_similarity(abstracts_embeddings, input_embedding.reshape(1, -1))
    top_similar_papers_indices = np.argsort(cosine_scores, axis=0)[-5:][::-1].flatten()
    papers_list = [titles[i] for i in top_similar_papers_indices]
    return papers_list

In [ ]:
# exampel usage 1: (use this paper as input (Attention is All you Need))
input_paper = input("Enter the title of any paper you like: ")
recommend_papers = recommendation(input_paper)


print("Recommended Papers:-->")
for paper in recommend_papers:
    print(paper)

Enter the title of any paper you like: Attention is All you Need
Recommended Papers:-->
Attention that does not Explain Away
Area Attention
Pay Attention when Required
Long Short-Term Attention
Attention as Activation


In [ ]:
input_abstracts = input("Enter the abstract or a bit of description of any paper you like: ")
recommend_papers = recommendation_abstracts(input_abstracts)
print("Recommended Papers:-->")
for paper in recommend_papers:
    print(paper)

Enter the title of any paper you like: Graph convolutional networks (GCNs) are powerful tools for graph-structured data. However, they have been recently shown to be vulnerable to topological attacks. To enhance adversarial robustness, we go beyond spectral graph theory to robust graph theory. 
Recommended Papers:-->
Power up! Robust Graph Convolutional Network via Graph Powering
A Targeted Universal Attack on Graph Convolutional Network
Adversarial Attacks and Defenses on Graphs: A Review, A Tool and Empirical Studies
Graph Structure Learning for Robust Graph Neural Networks
Understanding Structural Vulnerability in Graph Convolutional Networks
